In [1]:
model_name = '7b'
output_dir = 'OUTPUTS'
model_dir = 'models'

In [2]:
import util
from util import create_model_tokenizer

model, tokenizer = create_model_tokenizer(model_name, True, False)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/home/navarro/miniconda3/envs/py39/lib/python3.9/site-packages/transformers/utils/hub.py:374: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


In [3]:
import dataset_prompt

dataset = dataset_prompt.generate_dataset()

Map:   0%|          | 0/10356 [00:00<?, ? examples/s]

In [4]:
dataset

DatasetDict({
    train: Dataset({
        features: ['conversation', 'summary', 'text'],
        num_rows: 9320
    })
    test: Dataset({
        features: ['conversation', 'summary', 'text'],
        num_rows: 518
    })
    valid: Dataset({
        features: ['conversation', 'summary', 'text'],
        num_rows: 518
    })
})

In [5]:
dataset['train'][0]

{'conversation': 'Para que serve Slow K?',
 'summary': 'O Slow-K é um medicamento que trata e previne a da falta de potássio no organismo, devido a ingestão insuficiente pela alimentação ou por eliminação exagerada por doença, renal ou intestinal ou situações como diarreias, vômitos ou uso excessivo de laxantes.',
 'text': "[INST] <<SYS>>\nYou are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe.  Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.\n\nIf a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.\n<</SYS>>\nPara que serve Slow K?\n[/INST]\n\nO Slow-K é um medicamento que trata e previne a da falta de potássio no organismo, devido a ingestão 

In [6]:
model = util.prepare_model(model)
#model = util.get_peft_model(model, util.get_lora_config())

In [7]:
from transformers import AutoModelForCausalLM, AutoTokenizer, Trainer, TrainingArguments, DataCollatorForLanguageModeling
training_args = TrainingArguments(
    per_device_train_batch_size = 1, #The batch size per GPU (default: 8)
    gradient_accumulation_steps = 1,
    warmup_steps = 0, # Number of steps used for a linear warmup from 0 to learning_rate (default: 0)
    num_train_epochs=10,
   # weight_decay=0.1,
    learning_rate = 1e-3,
    fp16 = True, # Whether to use fp16 16-bit (mixed) precision training instead of 32-bit training.
    logging_steps = 1, # Number of update steps between two logs if logging_strategy="steps". Should be an integer or a float in range [0,1). 
    #If smaller than 1, will be interpreted as ratio of total training steps.
    overwrite_output_dir = True,
    evaluation_strategy = "epoch", #"The evaluation strategy to adopt during training. "
    save_strategy = "no",
    push_to_hub = False,
    output_dir = output_dir,
    report_to="tensorboard",
    optim="paged_adamw_8bit",
)

In [8]:
from trl import SFTTrainer
trainer_sft =  SFTTrainer(
    model = model,
    train_dataset=dataset['train'],
    eval_dataset=dataset['valid'],
    peft_config = util.get_lora_config(),
    dataset_text_field = "text",
    max_seq_length = 2048,
    tokenizer=tokenizer,
    args=training_args
)

Map:   0%|          | 0/9320 [00:00<?, ? examples/s]

Map:   0%|          | 0/518 [00:00<?, ? examples/s]

In [9]:
model.config.use_cache = False
trainer_sft.train()
model.config.use_cache = True

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


/home/navarro/miniconda3/envs/py39/lib/python3.9/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/home/navarro/miniconda3/envs/py39/lib/python3.9/site-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Epoch,Training Loss,Validation Loss
1,1.755900,0.749058
2,1.526900,0.749517
3,1.026500,0.740227
4,0.570200,0.692841
5,0.511400,0.682139
6,0.764500,0.650538
7,0.319400,0.614566
8,0.177500,0.593393
9,0.821700,0.581771
10,0.364200,0.581152


In [10]:
trainer_sft.save_model(f"{model_dir}/{model_name}")
tokenizer.save_pretrained(f"{model_dir}/{model_name}")

('models/7b/tokenizer_config.json',
 'models/7b/special_tokens_map.json',
 'models/7b/tokenizer.json')